The key thing I'll work towards here is a `Model` that has a dictionary of `params` and a `forward` method.

In [1]:
import inspect
from typing import List, NamedTuple, Callable, Optional, Union, Iterator, Dict

import numpy as np
np.set_printoptions(precision=4)

In [2]:
Arrayable = Union[float, list, np.ndarray]

Tensorable = Union['Tensor', float, np.ndarray]

def ensure_tensor(tensorable: Tensorable) -> 'Tensor':
    if isinstance(tensorable, Tensor):
        return tensorable
    else:
        return Tensor(tensorable)
    
def ensure_array(arrayable: Arrayable) -> np.ndarray:
    if isinstance(arrayable, np.ndarray):
        return arrayable
    else:
        return np.array(arrayable)

class Dependency(NamedTuple):
    tensor: 'Tensor'
    grad_fn: Callable[[np.ndarray], np.ndarray]
        
def collapse_sum(grad: np.ndarray,
                 t: 'Tensor') -> np.ndarray:

    # Sum out added dims
    ndims_added = grad.ndim - t.data.ndim
    for _ in range(ndims_added):
        grad = grad.sum(axis=0)

    # Sum across broadcasted (but non-added dims)
    for i, dim in enumerate(t.shape):
        if dim == 1:
            grad = grad.sum(axis=i, keepdims=True)
    
    return grad

In [3]:
class Tensor:
    
    def __init__(self,
                 data: np.ndarray,
                 depends_on: List[Dependency] = None,
                 no_grad: bool = False) -> None:
        self.data = ensure_array(data)
        self.depends_on = depends_on or []
        self.no_grad = no_grad
        self.shape = self.data.shape
        self.grad: Optional['Tensor'] = None
        if not self.no_grad:
            self.zero_grad()

    def __repr__(self) -> str:
        return f"Tensor({np.round(self.data, 4)})"
            
    def __add__(self, other: Tensorable) -> 'Tensor':
        """gets called if I do t + other"""
        return _add(self, ensure_tensor(other))

    def __radd__(self, other: Tensorable) -> 'Tensor':
        """gets called if I do other + t"""
        return _add(ensure_tensor(other), self)

    def __iadd__(self, other: Tensorable) -> 'Tensor':
        """when we do t += other"""
        self.data = self.data + ensure_tensor(other).data
        return self    

    def __isub__(self, other: Tensorable) -> 'Tensor':
        """when we do t -= other"""
        self.data = self.data - ensure_tensor(other).data
        return self
    
    def __imul__(self, other: Tensorable) -> 'Tensor':
        """when we do t *= other"""
        self.data = self.data * ensure_tensor(other).data
        return self

    def __mul__(self, other: Tensorable) -> 'Tensor':
        return _mul(self, ensure_tensor(other))

    def __rmul__(self, other: Tensorable) -> 'Tensor':
        return _mul(ensure_tensor(other), self)

    def __matmul__(self, other: Tensorable) -> 'Tensor':
        return _matmul(self, other)

    def __neg__(self) -> 'Tensor':
        return _neg(self)

    def __sub__(self, other: Tensorable) -> 'Tensor':
        return _sub(self, ensure_tensor(other))

    def __rsub__(self, other: Tensorable) -> 'Tensor':
        return _sub(ensure_tensor(other), self)
    
    def __getitem__(self, idxs) -> 'Tensor':
        return _slice(self, idxs)
    
    def concat(self, other: Tensorable) -> 'Tensor':
        return _concat(self, ensure_tensor(other))
    
    def repeat(self, repeats: int) -> 'Tensor':
        return _repeat(self, repeats)
    
    def zero_grad(self) -> None:
        self.grad = Tensor(np.zeros_like(self.data, dtype=np.float64),
                           no_grad = True)
    
    def sum(self) -> 'Tensor':
        return tensor_sum(self)
    
    def backward(self, grad: 'Tensor' = None) -> None:
        # backward mostly going to be called on the loss after calling "sum"
        if self.no_grad:
            return
        
        if self.shape == ():
            grad = Tensor(np.array(1.0))

        
        self.grad.data = self.grad.data + grad.data

        for dependency in self.depends_on:
            backward_grad = dependency.grad_fn(grad.data)
            dependency.tensor.backward(Tensor(backward_grad))

#### Backward functions

In [4]:
def _add(t1: Tensor, t2: Tensor) -> Tensor:

    def _forward(t1: Tensor, t2: Tensor) -> np.ndarray:
        return t1.data + t2.data

    def t1_grad(grad: np.ndarray) -> np.ndarray:

        grad = collapse_sum(grad, t1)

        return grad

    def t2_grad(grad: np.ndarray) -> np.ndarray:

        grad = collapse_sum(grad, t2)

        return grad

    data = _forward(t1, t2)
    depends_on = [
        Dependency(t1, t1_grad),
        Dependency(t2, t2_grad)
    ]
    return Tensor(data, depends_on)

def _mul(t1: Tensor, t2: Tensor) -> Tensor:

    def _forward(t1: Tensor, t2: Tensor) -> np.ndarray:
        return t1.data * t2.data

    def t1_grad(grad: np.ndarray) -> np.ndarray:
        grad = grad * t2.data
        grad = collapse_sum(grad, t1)

        return grad

    def t2_grad(grad: np.ndarray) -> np.ndarray:
        grad = grad * t1.data
        grad = collapse_sum(grad, t2)

        return grad

    data = _forward(t1, t2)
    depends_on = [
        Dependency(t1, t1_grad),
        Dependency(t2, t2_grad)
    ]
    return Tensor(data, depends_on)

def _matmul(t1: Tensor, t2: Tensor) -> Tensor:

    assert t1.shape[1] == t2.shape[0]

    def _forward(t1: Tensor, t2: Tensor) -> np.ndarray:
        return t1.data @ t2.data

    def t1_grad(grad: np.ndarray) -> np.ndarray:
        grad = grad @ t2.data.T

        return grad

    def t2_grad(grad: np.ndarray) -> np.ndarray:
        grad = t1.data.T @ grad

        return grad

    data = _forward(t1, t2)
    depends_on = [
        Dependency(t1, t1_grad),
        Dependency(t2, t2_grad)
    ]
    return Tensor(data, depends_on)

def _neg(t: Tensor) -> Tensor:

    def _forward(t: Tensor) -> np.ndarray:
        return -t.data

    def t_grad(grad: np.ndarray) -> np.ndarray:
        return -grad

    data = _forward(t)
    depends_on = [
        Dependency(t, t_grad),
    ]
    return Tensor(data, depends_on)

def _sub(t1: Tensor, t2: Tensor) -> Tensor:
    return t1 + -t2

def _slice(t: Tensor, idxs: slice) -> Tensor:

    def _forward(t: Tensor):
        return t.data[idxs]

    data = _forward(t)    
    
    def t_grad(grad: np.ndarray) -> np.ndarray:
        bigger_grad = np.zeros_like(data)
        bigger_grad[idxs] = grad
        return bigger_grad

    depends_on = [
        Dependency(t, t_grad),
    ]

    return Tensor(data, depends_on)

def tensor_sum(t: Tensor) -> Tensor:

    def _forward(t: Tensor):
        return t.data.sum()

    def t_grad(grad: np.ndarray) -> np.ndarray:
        return grad * np.ones_like(t.data)

    data = _forward(t)
    depends_on = [
        Dependency(t, t_grad),
    ]

    return Tensor(data, depends_on)        
        

#### Extra functions

In [5]:
def _concat(t1: Tensor, t2: Tensor) -> Tensor:
    
    assert t1.shape[0] == t2.shape[0],\
    "Concatenated Tensors must have the same shape along first dimension"
    
    def _forward(t1: Tensor, t2: Tensor):
        return np.concatenate([t1.data, t2.data], axis=1)    
    
    def t1_grad(grad: np.ndarray) -> np.ndarray:
        return grad[:,:t1.shape[1]]
    
    def t2_grad(grad: np.ndarray) -> np.ndarray:
        return grad[:,t1.shape[1]:]

    data = _forward(t1, t2)
    
    depends_on = [
        Dependency(t1, t1_grad),
        Dependency(t2, t2_grad)
    ]

    return Tensor(data, depends_on)

In [6]:
def _repeat(t: Tensor, repeats: int) -> Tensor:

    assert t.shape[0] == 1,\
    "Repeat operation should only be used on rows"
    
    def _forward(t: Tensor, repeats: int) -> np.ndarray:
        return np.repeat(t.data, repeats, axis=0)    
    
    def t_grad(grad: np.ndarray) -> np.ndarray:
        return grad.sum(axis=0)

    data = _forward(t, repeats)
    
    depends_on = [
        Dependency(t, t_grad)
    ]

    return Tensor(data, depends_on)

#### Test `concat`

In [7]:
# testing concat forward

a = np.random.randn(2, 3)
b = np.random.randn(2, 5)

c = np.concatenate([a, b], axis=1)
c.shape

(2, 8)

In [8]:
# concat grad calculations
a_grad = c[:,:a.shape[1]]
b_grad = c[:,a.shape[1]:]

In [9]:
print(a_grad.shape)
print(b_grad.shape)

(2, 3)
(2, 5)


In [10]:
# testing concat forward

aT = Tensor(a)
bT = Tensor(b)
c = aT.concat(bT)

d = c.sum()

d.backward()

In [11]:
print(aT.grad)
print(bT.grad)

Tensor([[1. 1. 1.]
 [1. 1. 1.]])
Tensor([[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]])


#### Test `repeat`

In [12]:
# testing repeat forward

a = np.random.randn(1, 3)

np.repeat(a, 2, axis=0)

array([[ 0.8046, -1.7214,  0.6557],
       [ 0.8046, -1.7214,  0.6557]])

In [13]:
# repeat backward

a = np.random.randn(1, 3)

a2 = np.repeat(a, 2, axis=0)

a2.sum(axis=0)

array([-3.1888, -2.8004,  1.0108])

In [14]:
# repeat with tensors

a = np.random.randn(1, 3)
b = np.random.randn(3, 3)

aT = Tensor(a)
bT = Tensor(b)
a2 = aT.repeat(3)
c = (a2 + bT).sum()
c.backward()

In [15]:
aT.grad

Tensor([[3. 3. 3.]])

#### Other classes

In [16]:
class Parameter(Tensor):
    def __init__(self, *shape) -> None:
        data = np.random.randn(*shape)
        super().__init__(data)

In [17]:
class Model:
    def parameters(self) -> Iterator[Parameter]:
        for name, value in inspect.getmembers(self):
            if isinstance(value, Parameter):
                yield value
            elif isinstance(value, Model):
                yield from value.parameters()

    def zero_grad(self):
        for parameter in self.parameters():
            parameter.zero_grad()

In [18]:
class SGD:
    def __init__(self, lr: float = 0.01) -> None:
        self.lr = lr

    def step(self, model: Model) -> None:
        for parameter in model.parameters():
            parameter -= parameter.grad * self.lr


#### Activations

In [19]:
def tanh(t: Tensor) -> Tensor:
    def _forward(t: Tensor):
        return np.tanh(t.data)

    data = _forward(t)
    
    def t_grad(grad: np.ndarray) -> np.ndarray:
        return grad * (1 - data * data)

    depends_on = [
        Dependency(t, t_grad)
    ]
    
    return Tensor(data, depends_on)

In [20]:
def sigmoid(t: Tensor) -> Tensor:
    
    def _forward(t: Tensor):
        return 1.0 / (1.0 + np.exp(-t.data))

    data = _forward(t)
    
    def t_grad(grad: np.ndarray) -> np.ndarray:
        return grad * data * (1.0 - data)

    depends_on = [
        Dependency(t, t_grad)
    ]
    
    return Tensor(data, depends_on)

### Boston data

In [21]:
from sklearn.datasets import load_boston

boston = load_boston()

data = boston.data
target = boston.target
features = boston.feature_names

from sklearn.preprocessing import StandardScaler
s = StandardScaler()
data = s.fit_transform(data)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=80718)

y_train, y_test = y_train.reshape((-1,1)), y_test.reshape((-1,1))

X_train, X_test, y_train, y_test = (Tensor(X_train, no_grad=True),
                                    Tensor(X_test, no_grad=True),
                                    Tensor(y_train, no_grad=True),
                                    Tensor(y_test, no_grad=True))

In [22]:
class BostonModel(Model):
    def __init__(self, 
                 num_hidden: int = 13,
                 seed: int = 1) -> None:
        np.random.seed(seed)
        self.w1 = Parameter(13, num_hidden)
        self.b1 = Parameter(num_hidden)

        self.w2 = Parameter(num_hidden, 1)
        self.b2 = Parameter(1)

    def predict(self, inputs: Tensor) -> Tensor:
        # inputs will be (batch_size, 13)
        x1 = inputs @ self.w1 + self.b1  # (batch_size, num_hidden)
        x2 = tanh(x1)                    # (batch_size, num_hidden)
        x3 = x2 @ self.w2 + self.b2      # (batch_size, 4)

        return x3

### Train model

In [23]:
optimizer = SGD(lr=0.001)
batch_size = 32
model = BostonModel(seed=112418)
train_size = X_train.shape[0]

for epoch in range(10):
    epoch_loss = 0.0

    for start in range(0, train_size, batch_size):
        end = start + batch_size

        model.zero_grad()

        inputs = X_train[start:end]
        inputs.no_grad = True

        predicted = model.predict(inputs)
        actual = y_train[start:end]
        actual.no_grad = True

        errors = predicted - actual
        loss = (errors * errors).sum()
        loss.backward()
        optimizer.step(model)
        
        
        # test predictions
    predicted = model.predict(X_test)
    errors = predicted - y_test
    loss = (errors * errors).sum()
        
    print(epoch, loss)

0 Tensor(10929.9298)
1 Tensor(5931.3808)
2 Tensor(5032.0439)
3 Tensor(4765.6974)
4 Tensor(4596.3881)
5 Tensor(4585.5644)
6 Tensor(4554.1706)
7 Tensor(4486.7719)
8 Tensor(4400.3648)
9 Tensor(4251.6124)


# Simple Tensor Examples 

### Simple forward with addition

```
a 
 \
  c - s
 /
b
```

In [24]:
np.random.seed(112518)
a = Tensor(np.random.randn(2,2))
b = Tensor(np.random.randn(2,2))
c = a + b
s = c.sum()

print(a.grad)
s.backward()
print(a.grad)

Tensor([[0. 0.]
 [0. 0.]])
Tensor([[1. 1.]
 [1. 1.]])


### Simple forward with branching

```
a 
 \
  c1 
 /  \
b
      s
a 
 \   /
  c2 
 /
b 
```

In [25]:
np.random.seed(112518)
a = Tensor(np.random.randn(2,2))
b = Tensor(np.random.randn(2,2))
c1 = a + b
c2 = a * b
s = (c1 + c2).sum()

print(a.grad)
s.backward()
print(a.grad)
print()
print("a:", a)
print("s:", s)

Tensor([[0. 0.]
 [0. 0.]])
Tensor([[0.6462 2.4838]
 [1.0027 1.1719]])

a: Tensor([[ 2.1617 -0.4313]
 [ 0.3818  0.7105]])
s: Tensor(2.8455)


This implies that increasing `a[0][0]` by 0.1 from `2.1617` to `2.2617` would increase `s` from `2.8455` to:

In [26]:
2.8455 + 0.1 * 0.6462

2.91012

Let's check:

In [27]:
def check_sum(a_first_val):
    
    np.random.seed(112518)
    a = np.random.randn(2,2)
    b = np.random.randn(2,2)
    a2 = a.copy()
    a2[0][0] = a_first_val
    c1 = a2 + b
    c2 = a2 * b
    return (c1 + c2).sum()

In [28]:
print(check_sum(2.1617))
print(check_sum(2.2617))

2.8454767470862894
2.910093928197016


Works!

### Using the same array multiple times

In [29]:
w = Tensor(np.random.randn(2,2))
a = Tensor(np.random.randn(2,2))
b = Tensor(np.random.randn(2,2))
c = w + a
s = (w * b).sum()

s.backward()

print("w:", w)
print("s:", s)
print("w.grad:", w.grad)

w: Tensor([[-0.0683  0.6959]
 [ 1.4791  0.5237]])
s: Tensor(-1.2067)
w.grad: Tensor([[-0.5692 -0.3029]
 [-0.0847 -1.7367]])


In [30]:
-0.0683 + 0.1

0.031700000000000006

This implies that increasing `w[0][0]` by 0.1 from `-0.0683` to `0.0317` would increase `s` from `-1.2067` to:

In [31]:
-1.2067 + 0.1 * -0.5692

-1.2636200000000002

In [32]:
def check_sum(w_first_val):
    
    w2 = w.data.copy()
    w2[0][0] = w_first_val
    c.data = w2 + a.data
    return (w2 * b.data).sum()


In [33]:
print(check_sum(-0.0683))
print(check_sum(0.0317))

-1.206689062693263
-1.263613361478857


### Adding bias multiple times

Will mimic adding hidden state multiple times in an LSTM.

### Ultimate goal:

```python
def lstm_node(inputs: Tensor, 
              hiddens: Tensor, 
              cells: Tensor, 
              params: Dict[str, Tensor]):
    
    assert input.shape[0] == hidden.shape[0] == cell.shape[0]
    
    Z = inputs.concat(hidden)
    
    forget = sigmoid(Z @ params['Wf'] + params['Bf'])

    ingate = sigmoid(Z @ params['Wi'] + params['Bi'])
    
    outgate = sigmoid(Z @ params['Wo'] + params['Bo'])
    
    change = tanh(Z @ params['Wc'] + params['Bc'])
    
    cells = cells * forget + ingate * change
    
    hiddens = outgate * tanh(cells)
    
    outputs = hidden @ params['Wv'] + params['Bv']
    
    return outputs, hiddens, cells
```

In [34]:
def init_params(batch_size: int,
                state_size: int,
                vocab_size: int) -> Dict[str, Parameter]:
    
    params = {}
    params['Wf'] = Parameter(state_size + vocab_size, state_size)
    params['Wi'] = Parameter(state_size + vocab_size, state_size)
    params['Wo'] = Parameter(state_size + vocab_size, state_size)
    params['Wc'] = Parameter(state_size + vocab_size, state_size)
    params['Wv'] = Parameter(state_size, vocab_size)
    
    params['Bf'] = Parameter(state_size)
    params['Bi'] = Parameter(state_size)
    params['Bo'] = Parameter(state_size)
    params['Bc'] = Parameter(state_size)
    params['Bv'] = Parameter(vocab_size)
    
    return params

In [35]:
# constants
batch_size = 12
state_size = 20
vocab_size = 30

np.random.seed(112518)

# initial data
h_init = Tensor(np.random.randn(1, state_size))
c_init = Tensor(np.random.randn(1, state_size))
inputs = Tensor(np.random.randn(batch_size, vocab_size))

# initialize params
params = init_params(batch_size, state_size, vocab_size)

# repeat cell and hidden
hiddens = h_init.repeat(batch_size)
cells = c_init.repeat(batch_size)

In [36]:
def lstm_node(inputs: Tensor, 
              hiddens: Tensor, 
              cells: Tensor, 
              params: Dict[str, Parameter]):

    assert inputs.shape[0] == hiddens.shape[0] == cells.shape[0]

    Z = inputs.concat(hiddens)

    forget = sigmoid(Z @ params['Wf'] + params['Bf'])

    ingate = sigmoid(Z @ params['Wi'] + params['Bi'])

    outgate = sigmoid(Z @ params['Wo'] + params['Bo'])

    change = tanh(Z @ params['Wc'] + params['Bc'])

    cells = cells * forget + ingate * change

    hiddens = outgate * tanh(cells)

    outputs = hiddens @ params['Wv'] + params['Bv']

    return outputs, hiddens, cells

In [37]:
o_out, h_out, c_out = lstm_node(inputs, hiddens, cells, params)

In [38]:
o_out.shape # (batch_size, vocab_size)

(12, 30)

In [39]:
s = o_out.sum()
s.backward()

In [40]:
s

Tensor(70.9797)

In [41]:
print(h_init)
print(h_init.grad)

Tensor([[ 2.1617 -0.4313  0.3818  0.7105 -0.3538  1.4838  0.0027  0.1719 -0.0683
   0.6959  1.4791  0.5237  1.0117  0.0821 -0.2638 -1.4929 -0.5692 -0.3029
  -0.0847 -1.7367]])
Tensor([[  5.8457  -3.3987   3.0234   9.0145   7.7679  -2.7426   3.7362 -25.35
   -7.5784 -19.3007  17.2762   8.8289 -21.3072 -16.8012 -21.413   19.6036
    1.4624  10.1015   4.6027   2.511 ]])


This implies that if I increase `h_init[0]` from `2.1617` to `2.2617`, `s` will change to:

In [49]:
70.9797 + 0.1 * 5.8457

71.56427

In [43]:
def lstm_node_np(inputs: np.ndarray, 
              hiddens: np.ndarray, 
              cells: np.ndarray, 
              params: Dict[str, np.ndarray]):

    assert inputs.shape[0] == hiddens.shape[0] == cells.shape[0]

    def _sigmoid(arr: np.ndarray) -> np.ndarray:
        return 1.0 / (1.0 + np.exp(-arr))
        
    Z = np.concatenate([inputs, hiddens], axis=1)

    forget = _sigmoid(Z @ params['Wf'].data + params['Bf'].data)

    ingate = _sigmoid(Z @ params['Wi'].data + params['Bi'].data)

    outgate = _sigmoid(Z @ params['Wo'].data + params['Bo'].data)

    change = np.tanh(Z @ params['Wc'].data + params['Bc'].data)

    cells = cells * forget + ingate * change

    hiddens = outgate * np.tanh(cells)

    outputs = hiddens @ params['Wv'].data + params['Bv'].data

    return outputs, hiddens, cells

In [44]:
h_init_np = h_init.data
print(h_init_np)
h_init_np[0][0] += 0.1
print(h_init_np)

[[ 2.1617 -0.4313  0.3818  0.7105 -0.3538  1.4838  0.0027  0.1719 -0.0683
   0.6959  1.4791  0.5237  1.0117  0.0821 -0.2638 -1.4929 -0.5692 -0.3029
  -0.0847 -1.7367]]
[[ 2.2617 -0.4313  0.3818  0.7105 -0.3538  1.4838  0.0027  0.1719 -0.0683
   0.6959  1.4791  0.5237  1.0117  0.0821 -0.2638 -1.4929 -0.5692 -0.3029
  -0.0847 -1.7367]]


In [45]:
hiddens = np.repeat(h_init_np, batch_size, axis=0)
hiddens.shape

(12, 20)

In [46]:
out_np, h_np, c_np = lstm_node_np(inputs.data, 
                                  hiddens,
                                  cells.data,
                                  params)

In [47]:
out_np.shape

(12, 30)

In [48]:
out_np.sum()

71.57204981645923

Close!